In [2]:
import numpy as np

DEBUG = True

def tiempom_espera5(llegada, activo, nstream, nterm):
    p0 = 0
    z = activo / llegada
    f = []
    fp = []
    pn = []
    p1 = []

    f0 = nterm * activo / llegada
    for i in range(nterm):
        if i <= nstream - 1:
            f.append((nterm - i) * activo / (llegada * (i + 1)))
        else:
            f.append((nterm - i) * activo / (llegada * nstream))

    for i in range(1, nterm + 1):
        vect = np.prod(f[:i])
        fp.append(vect)

    p0 = 1 / (1 + sum(fp))

    for i in range(1, nterm + 1):
        vect = np.prod(f[:i])
        pn.append(vect * p0)

    prob_ocupado = sum(pn[nstream - 1:])

    for i in range(nterm):
        p1.append((i + 1) * pn[i])

    L = sum(p1)
    Lq = max(0, L - (z * (nterm - L)))
    Wq = (Lq * llegada) / (nterm - L)
    tasa = (nterm - L) / llegada
    ro = tasa * activo / nstream

    return Lq, Wq, tasa, prob_ocupado, ro

def findOptConfPv3(nstages, thC, thG, nc):
    confP = ['0'] * nstages
    botlC, stC = min((val, idx) for (idx, val) in enumerate(thC))
    botlG, stG = min((val, idx) for (idx, val) in enumerate(thG))
    
    if botlC < botlG:
        confP[stC] = '1'
        stageBotl = stC
        bottleneck_device = 'CPU'
    else:
        confP[stG] = '0'
        stageBotl = stG
        bottleneck_device = 'GPU'
        
    TserCP = np.zeros(2**nstages)
    TserGP = np.zeros(2**nstages)
    TserCS = np.zeros(2**nstages)
    TserGS = np.zeros(2**nstages)

    p = np.ones(2**nstages)
    Sdev = -np.ones(2**nstages)
    rhoG = np.ones(2**nstages)
    rhoC = np.ones(2**nstages)
    lambda_vals = np.zeros(2**nstages)

    for i in range(1, 2**nstages):
        if i == 2**nstages - 1:
            for k in range(nstages):
                TserGP[i] += 1 / thG[k]
                TserCP[i] += 1 / (thC[k] / nc)
            lambda_vals[i] = 1 / TserGP[i] + (1 / TserCP[i]) * nc
        else:
            conf = bin(i)[2:].zfill(nstages)
            if conf[stageBotl] == confP[stageBotl]:
                for k in range(nstages):
                    if conf[k] == '1':
                        TserGP[i] += 1 / thG[k]
                        TserCS[i] += 1 / (thC[k] / nc)
                    else:
                        TserCP[i] += 1 / (thC[k] / nc)
                        TserGS[i] += 1 / thG[k]
                if TserGP[i] == 0 or TserCP[i] == 0:
                    lambda_vals[i] = 0
                    continue
                
                lambdaGP = 1 / TserGP[i]
                lambdaCP = (1 / TserCP[i]) * nc
                lambdaG = lambdaGP
                lambdaC = lambdaCP

                if DEBUG:
                    print(f"Conf {i}: TserGP[i]={TserGP[i]}, TserCP[i]={TserCP[i]}")

                if lambdaGP < 0.9 * lambdaCP:
                    rhoC[i] = ((nc * TserGP[i] + TserCS[i]) / (TserGP[i] * TserCS[i])) * ((TserCP[i] * TserCS[i]) / (TserCP[i] + TserCS[i])) * (1 / nc)
                    cP = max(round(nc * rhoC[i]), 1)
                    cS = nc - cP
                    if rhoC[i] < 0.8:
                        Sdev[i] = 0
                        lambdaG = lambdaGP + ((1 / TserCS[i]) * nc) * (1 - rhoC[i])
                        lambdaC = lambdaCP * rhoC[i]
                        p[i] = lambdaGP / lambdaG
                else:
                    if lambdaCP < 0.9 * lambdaGP:
                        rhoG[i] = ((nc * TserGS[i] + TserCP[i]) / (TserGS[i] * TserCP[i])) * ((TserGP[i] * TserGS[i]) / (TserGP[i] + TserGS[i]))
                        if rhoG[i] < 0.8:
                            Sdev[i] = 1
                            lambdaG = lambdaGP * rhoG[i]
                            lambdaC = lambdaCP + (1 / TserGS[i]) * (1 - rhoG[i])
                            p[i] = lambdaCP / lambdaC
                
                lambda_vals[i] = min(lambdaG, lambdaC)
            else:
                lambda_vals[i] = 0

    lambdaOpt = []
    confOptP = []
    confOptS = []
    lambdae = []
    ntokens = []
    coresP = []
    coresS = []
    
    for k in range(5):
        if max(lambda_vals) == 0:
            break
        max_lambda = max(lambda_vals)
        id_opt = np.argmax(lambda_vals)
        lambda_vals[id_opt] = 0
        
        lambdaOpt.append(max_lambda)
        confOptP.append(bin(id_opt)[2:].zfill(nstages))
        confOptS.append(Sdev[id_opt])

        if DEBUG:
            print(f"Top {k+1} confOptP= {confOptP[-1]} with lambdaOpt= {lambdaOpt[-1]}")
        
        romax = 0.95
        if id_opt == 2**nstages - 1:
            Tarrive = TserGP[id_opt]
        else:
            if confOptS[-1] == 0:
                Tarrive = 1 / (p[id_opt] * lambdaOpt[-1])
            elif confOptS[-1] == 1:
                Tarrive = 1 / lambdaOpt[-1]
                romax = 0.95 * rhoG[id_opt]
            else:
                Tarrive = 1 / lambdaOpt[-1]
        
        TserG = TserGP[id_opt]
        c = 1
        NGP = 0
        roG = 0
        while roG < romax:
            NGP += 1
            LqG, WqG, lambdaeGP, prob_ocupadoG, roG = tiempom_espera5(Tarrive, TserG, c, NGP)
        
        if DEBUG:
            print(f"Primary GPU path: M/M/1/NGP/NGP with NGP= {NGP} and lambdaeGP= {lambdaeGP}")
        
        romax = 0.95
        if id_opt == 2**nstages - 1:
            Tarrive = TserCP[id_opt]
            cP = nc
        else:
            if confOptS[-1] == 0:
                Tarrive = 1 / lambdaOpt[-1]
                romax = 0.95 * rhoC[id_opt]
            elif confOptS[-1] == 1:
                Tarrive = 1 / (p[id_opt] * lambdaOpt[-1])
            else:
                Tarrive = 1 / lambdaOpt[-1]
            cP = max(round(nc * rhoC[id_opt]), 1)

        TserC = TserCP[id_opt]
        c = nc
        NCP = 0
        roC = 0
        while roC < romax:
            NCP += 1
            LqC, WqC, lambdaeCP, prob_ocupadoC, roC = tiempom_espera5(Tarrive, TserC, c, NCP)
        
        if DEBUG:
            print(f"Primary CPU path: M/M/c/NCP/NCP with NCP= {NCP} and lambdaeCP= {lambdaeCP}")
        
        if confOptS[-1] == 1:
            Tarrive = 1 / ((1 - p[id_opt]) * lambdaOpt[-1])
            romax = 0.95 * (1 - rhoG[id_opt])
            TserG = TserGS[id_opt]
            c = 1
            NGS = 0
            roG = 0
            while roG < romax:
                NGS += 1
                LqG, WqG, lambdaeGS, prob_ocupadoG, roG = tiempom_espera5(Tarrive, TserG, c, NGS)
            
            if DEBUG:
                print(f"Secondary GPU path: M/M/1/NGS/NGS with NGS= {NGS} and lambdaeGS= {lambdaeGS}")
        else:
            NGS = 0
            lambdaeGS = 0

        if confOptS[-1] == 0:
            Tarrive = 1 / ((1 - p[id_opt]) * lambdaOpt[-1])
            romax = 0.95 * (1 - rhoC[id_opt])
            TserC = TserCS[id_opt]
            cS = nc - cP
            c = nc
            NCS = 0
            roC = 0
            while roC < romax:
                NCS += 1
                LqC, WqC, lambdaeCS, prob_ocupadoC, roC = tiempom_espera5(Tarrive, TserC, c, NCS)
                if cS == 0:
                    break
            
            if DEBUG:
                print(f"Secondary CPU path: M/M/c/NCS/NCS with NCS= {NCS} and lambdaeCS= {lambdaeCS}")
        else:
            NCS = 0
            cS = 0
            lambdaeCS = 0

        if id_opt == 2**nstages - 1:
            lambdae_val = lambdaeGP + lambdaeCP
        else:
            lambdae_val = min(lambdaOpt[-1], min(lambdaeGP + lambdaeCS, lambdaeCP + lambdaeGS))
        
        lambdae.append(lambdae_val)
        coresP.append(cP)
        coresS.append(cS)
        ntokens.append(NGP + NCP + NGS + NCS)

    return lambdaOpt, confOptP, confOptS, lambdae, ntokens, coresP, coresS

def compute_and_print_results(label, nstages, thC, thG, nc, thserial, using_sycl=False):
    print(f"{label} Computation")
    lambdaOpt, confOptP, confS, lambdae, ntokens, cP, cS = findOptConfPv3(nstages, thC, thG, nc)

    confFinal = []
    speedup = []

    for k in range(len(lambdaOpt)):
        SP = 'C' if confS[k] == 0 else ('G' if confS[k] == 1 else 'E')
        confFinal.append([confOptP[k], SP])
        speedup.append(lambdae[k] / thserial if lambdae[k] > 0 else 0)

    # Ordenar las configuraciones por lambdae de mayor a menor
    sorted_indices = np.argsort(lambdae)[::-1]
    lambdaOpt = [lambdaOpt[i] for i in sorted_indices]
    confOptP = [confOptP[i] for i in sorted_indices]
    confS = [confS[i] for i in sorted_indices]
    lambdae = [lambdae[i] for i in sorted_indices]
    ntokens = [ntokens[i] for i in sorted_indices]
    cP = [cP[i] for i in sorted_indices]
    cS = [cS[i] for i in sorted_indices]
    confFinal = [confFinal[i] for i in sorted_indices]
    speedup = [speedup[i] for i in sorted_indices]

    print(f"confFinal_{label.lower()}:")
    for conf in confFinal:
        print(conf)
    print(f"\nlambdae_{label.lower()}: {lambdae}")
    print(f"ntokens_{label.lower()}: {ntokens}")
    print(f"speedup_{label.lower()}: {speedup}")
    print(f"cP_{label.lower()}: {cP}")
    print(f"cS_{label.lower()}: {cS}\n\n")

# Constant values
thserial = 0.49
nstages = 3
nc = 8

# Throughputs
thC_cpp = [9, 465.7, 2.7]
thG_cpp = [19.2, 478, 5.9]
thC_sycl = [28.9, 1893, 4.6]
thG_sycl = [19.2, 478, 5.9]
thC_avx = [19.1, 670, 24.6]
thG_avx = [19.2, 478, 5.9]
thC_simd = [44.6, 661.1, 25.3]
thG_simd = [19.2, 478, 5.9]

# Perform computations
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("AlderLake 4K SIMD")
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
compute_and_print_results("SIMD", nstages, thC_simd, thG_simd, nc, thserial)

print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("C++ Computation")
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
compute_and_print_results("C++", nstages, thC_cpp, thG_cpp, nc, thserial)

print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("SYCL Computation")
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
compute_and_print_results("SYCL", nstages, thC_sycl, thG_sycl, 1, thserial, using_sycl=True)

print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("AVX Computation")
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
compute_and_print_results("AVX", nstages, thC_avx, thG_avx, nc, thserial)


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
AlderLake 4K SIMD
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
SIMD Computation
Conf 2: TserGP[i]=0.0020920502092050207, TserCP[i]=0.495577730906255
Conf 4: TserGP[i]=0.052083333333333336, TserCP[i]=0.32830657731185836
Conf 6: TserGP[i]=0.054175383542538355, TserCP[i]=0.31620553359683795
Top 1 confOptP= 010 with lambdaOpt= 20.462719312824657
Primary GPU path: M/M/1/NGP/NGP with NGP= 1 and lambdaeGP= 19.622690830348187
Primary CPU path: M/M/c/NCP/NCP with NCP= 9 and lambdaeCP= 15.472729952129772
Secondary GPU path: M/M/1/NGS/NGS with NGS= 3 and lambdaeGS= 4.204678758713228
Top 2 confOptP= 111 with lambdaOpt= 20.228929857259637
Primary GPU path: M/M/1/NGP/NGP with NGP= 4 and lambdaeGP= 4.402150452948235
Primary CPU path: M/M/c/NCP/NCP with NCP= 18 and lambdaeCP= 15.235923506396487
Top 3 confOptP= 100 with lambdaOpt= 19.2
Primary GPU path: M/M/1/NGP/NGP with NGP= 4 and lambdaeGP= 18.904615